# Qué hay acá?

Esta notebook será usada para obtener los tweets de cuentas de políticos en twitter, se mostrará una descripción de los datos obtenidos

In [13]:
import requests
import os
import json
import pandas as pd
#import csv

#import datetime
#import dateutil.parser
#import unicodedata

import time

### Obteniendo tweets

In [5]:
bearer_token = os.environ.get("TOKEN")

In [1]:
# LuchoXBolivia     -> 1222273871488917504
# LuisFerCamachoV   -> 1133033369850339328
# carlosdmesag      -> 260852828

In [ ]:
fields = ['author_id', 'created_at', 'id', 'text', 'referenced_tweets', 'in_reply_to_user_id']

my_path = "../data/raw/mesa.json"
my_path_excel = "../data/raw/mesa.xlsx"
cont = 0

df = pd.DataFrame(columns=fields)

#for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200):
for tweet in tweepy.Paginator(client.get_users_tweets, id=260852828,
                            tweet_fields=fields,
                            end_time=datetime.datetime.now() - datetime.timedelta(days=7),
                            start_time=datetime.datetime.now() - datetime.timedelta(days=14),
                            expansions='referenced_tweets.id',
                            max_results=100).flatten(limit=1000000):
    cont+=1
    
    if cont % 100 == 0:
        print(cont)

    #try:
    # if tweet.reference_tweets has length 0 print cont
    if tweet.referenced_tweets:
        # if is a retweet get the full text of original tweet, else get the tweet itself
        #if tweet.referenced_tweets[0]['type'] == 'retweeted':
        if tweet.referenced_tweets[0]['type'] == 'retweeted':
            #print('retweet' + str(cont))
            original_tweets = client.get_tweets(tweet.referenced_tweets[0]['id'])
            for or_tweet in original_tweets:
                text = or_tweet[0]
                #print(text)
                break
            # get full text
            #text = original_tweet.text

            ndict = {'author_id': [str(tweet.author_id)],
                'created_at': [str(tweet.created_at)],
                'id': [str(tweet.id)],
                'text': [str(text)],
                'referenced_tweets': [str(tweet.referenced_tweets)],
                'in_reply_to_user_id': [str(tweet.in_reply_to_user_id)]}

        else:
            #print('no retweet' + str(cont))
            ndict = {'author_id': [str(tweet.author_id)],
                'created_at': [str(tweet.created_at)],
                'id': [str(tweet.id)],
                'text': [str(tweet.text)],
                'referenced_tweets': [str(tweet.referenced_tweets)],
                'in_reply_to_user_id': [str(tweet.in_reply_to_user_id)]}
    else:
        #print('no referenced_tweets' + str(cont))
        ndict = {'author_id': [str(tweet.author_id)],
            'created_at': [str(tweet.created_at)],
            'id': [str(tweet.id)],
            'text': [str(tweet.text)],
            'referenced_tweets': [str(tweet.referenced_tweets)],
            'in_reply_to_user_id': [str(tweet.in_reply_to_user_id)]}
    #except:
        #ndict = {'author_id': ['error'], 'created_at': ['error'], 'id': ['error'], 'text': ['error'], 'referenced_tweets': ['error'], 'in_reply_to_user_id': ['error']}

    df = df.append(pd.DataFrame(ndict), ignore_index=True)

#expor to json
df.to_json(my_path, orient='records', lines=True, force_ascii=False)
print(df.shape)
df.head()

### Exportaciones

In [ ]:
df.info()

In [ ]:
# convert column 'created_at' to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
# export to json
df.to_json(my_path, orient='records', lines=True, force_ascii=False)

In [ ]:
df.head()

In [ ]:
# describe
df.describe()

In [ ]:
# export to excel
df.to_excel(my_path_excel, index=False)